"Descent into Collective Entanglement"

This thesis work is to explore the possibility of faster calculations for calculating "Collectibility" for a given density matrix for a given multipartite system. In this work, we mainly focus on the 3*3 system.

We we wilm be trying out gradient descent algorithms for the purpose of finding the optimum "separable state set" to get a minimum value of collectibility. This minima, in return, will indicate if the system is entangled.

We will start with testing on known classes of density matrices (eg, werner states) which have known bounds as a sanity check of the approach.

Steps - 
1. Function working with unitary matrix. It will have a function to calculate random unitary matrix using Zyczkowski's method. ✅
2. Moreover, it needs to calculate the partial differential of U wrt an individual parameter. And then return the gradient for the whole parameters vector. ✅
3. Decide on the step size.✅
4. Work on generating density matrices - Random Density matrices and werner states.✅
5. Work out the collectibility function for given parameters.✅
6. After calling the gradient calculating function and deciding on the step size, we calculate the value of Collectibility for the next step.✅
7. We find the local minima, and do this procedure more times, to search fort he global minima.
8. Visualize the results for plotting graphs and getting a visual sense of the work.
9. Compare with brute force methods by searching for the optimum solution (minima) by random search.

In [8]:
# Library imports

import random
import cmath,math
from pprint import pprint
import copy
import time
import itertools
import sys

#numpy and matplot
import matplotlib.pyplot as plt
import numpy as np

import time

#Qutip imports
from qutip import (Qobj, about, basis, coherent, coherent_dm, create, 
                  destroy,expect, fock, fock_dm, mesolve, qeye, sigmax, 
                  sigmay,sigmaz, tensor, thermal_dm,dimensions,
                  rand_dm,rand_unitary,partial_transpose,
                  hadamard_transform,projection,phase_basis)

import qutip

from functools import reduce

In [9]:
# List of intial states - 

#Random rho generator - using banacki's method (and maybe using Qobj)
def Mutually_Orthogonal_state_generator(system):
    '''
    Initially, the assumption is that all Hilbert spaces of the subsystem are of 
    the same dimension.
    Should I pad the lower dimension subsystems with zero columns??
    '''
    N = min(system)
    MO_Basis_set = []
    for subsys_dim in system:
        MO_Basis_subset = [basis(subsys_dim, n) for n in range(subsys_dim)]
        MO_Basis_set.append(MO_Basis_subset)
    return MO_Basis_set

def Seperable_state_set_generator(MO_Basis_set):
    #Initially, the assumption is that all Hilbert spaces of the subsystem are of 
    # the same dimension.
    
    Seperable_basis_set = []
    N = max([len(subsys) for subsys in MO_Basis_set])
    for n in range(N): 
        Seperable_basis_set.append(tensor([subsys[n] for subsys in MO_Basis_set]))
    return Seperable_basis_set

system = [3,3]        
Sep_state_set = Seperable_state_set_generator(Mutually_Orthogonal_state_generator(system))
print(Sep_state_set)

[Quantum object: dims = [[3, 3], [1, 1]], shape = (9, 1), type = ket
Qobj data =
[[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], Quantum object: dims = [[3, 3], [1, 1]], shape = (9, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]], Quantum object: dims = [[3, 3], [1, 1]], shape = (9, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]]]


In [10]:
# Generating rho (random and werner)
# Generating Random Density Matrices - few options - 

# 1. Random Density matrix using built in Qutip functions - 
rand_dm(size)

# 2. Generating Werner State

def Probability_generator(n):
    '''
    Generates list of Probabilites with n elements, for by partitioning 1 with each step. 
    Hence, adding up to 1.
    '''
    Probs = []
    limit = 1
    for i in range(n-1):
        p = random.uniform(0,limit)
        limit = limit - p
        Probs.append(p)
    Probs.append(limit)
    return Probs

def Probability_generator1(n):
    '''
    Generates list of Probabilites with n elements, by first generating n-1 probabilities.
    Then sorting them, add 0 and 1 to the list and we get list of Pobabilities that add up to 1 by taking difference 
    between elements of the previous list. Suggested by Dr. Banacki.
    '''
    
    Probs = []
    for i in range(n-1):
        R = random.uniform(0,1)
        if R not in [0,1]:
            Probs.append(R)
    Probs = sorted(Probs+[0,1])
    Probabilities = []
    for j in range(n):
        Probabilities.append(Probs[j+1]-Probs[j]) 
    return Probabilities

def Simplex_generator(n):
    P = Probability_generator1(n)
    simplex = np.identity(n)
    for i in range(n):
        simplex[i,i] = P[i]
    simplex = Qobj(simplex)
    return simplex

def Amplitude_Generator(Probs):
    for i in range(len(Probs)):
        Probs[i] = math.sqrt(Probs[i])
    return Probs

def Normalized_pure_state_generator(dim):
    simplex = Amplitude_Generator(Probability_generator1(dim))
    Q = [basis(dim, n) for n in range(dim)]
    NPS = []
    for i in range(dim):
        W = simplex[i]*Qobj(dimensions.collapse_dims_oper(tensor([Q[i],Q[i]])))
        NPS.append(W)
    NPS = sum(NPS)
    return(NPS)

def Arbitrary_Mixed_State(n):
    simplex = Simplex_generator(n)
    U = rand_unitary(n)
    AMS = U*simplex*U.dag()
    return AMS 

def Reshaping_Qobj(Q,subsys):
    K = len(subsys)
    N = np.prod(np.array(subsys))
    if N != Q.shape[0]:
        raise ValueError("Product of dimensions of subsystem and Total dimension of Qobj dont match")
    Q.dims = [subsys for i in range(len(subsys))]
    return Q
    
def Werner_Density_Matrix(dim,alpha):
    '''
    For an dim*dim system, Werner state is given as -
    l
    NPS = Normalized Pure State
    '''
    NPS = Normalized_pure_state_generator(dim)
    UV = tensor(rand_unitary(dim),rand_unitary(dim))
    print(UV)
    rho = alpha*UV*NPS*NPS.dag()*UV.dag() + (1-alpha)/(dim**2)*qeye(dim**2)
    return rho

# print(Werner_Density_Matrix(3,0.5))
AMS = Arbitrary_Mixed_State(6)
AMS = Reshaping_Qobj(AMS,[2,3])
print(AMS)
# partial_transpose(AMS, [False,True]).eigenenergies()

NameError: name 'size' is not defined

In [11]:
# Helper function 1
'''
Helper functions - 
all the function that can be used across multiple modules 
and repeatedly often.
''' 

def generate_Omega(delta):
    Omega = random.uniform(0,delta)
    # For now, excluding Omega==0 for getting sensible dphi_dOmega
    while Omega==(0 or 1):
        print(Omega)
        Omega = random.uniform(0,delta)
    return Omega

def generate_alpha(delta):
    alpha = random.uniform(0,2*cmath.pi*delta)
    return alpha

def generate_phi(r,s,delta,Omega):
    phi = math.asin(Omega**(1/(2*r)))
    return phi

def generate_psi(r,s,delta):
    psi = random.uniform(0,2*cmath.pi*delta)
    return psi

def generate_chi(r,s,delta):
    chi = random.uniform(0,2*cmath.pi*delta)
    return chi

def para_i(r,s):
    return (s**2-2*r-1, s**2-2*r, (s+1)**2-2*(s+1)) 

In [12]:
# Helper functions 2

def Series_multiply(lst, order = 'normal'):
    Multiplied_list = []
    M = 1
    
    if order=='right':
        lst1 = lst.copy()
        lst1.reverse()
        
        for ls in lst1:
            if isinstance(ls,np.ndarray) and isinstance(M,np.ndarray):
                M = np.matmul(ls,M)
            else:
                M = ls*M
            Multiplied_list.append(M)

        Multiplied_list.reverse()
        
    elif order == 'left':
        for ls in lst:
            if isinstance(ls,np.ndarray) and isinstance(M,np.ndarray):
                M = np.matmul(M,ls)
            else:
                M = M*ls
            Multiplied_list.append(M)
            
    else:
        for ls in lst:
            if isinstance(ls,np.ndarray) and isinstance(M,np.ndarray):
                M = np.matmul(M,ls)
            else:
                M = M*ls
        return M
    
    return Multiplied_list

In [13]:
def tensor_product(matrices):
    if len(matrices) < 2:
        raise ValueError("At least two matrices are required.")

    return reduce(np.kron, matrices)

def tensor_product_left(tensors):
    if len(tensors) < 2:
        raise ValueError("At least two tensors are required.")

    result = [tensors[0]]
    for i in range(1, len(tensors)):
        result.append(np.kron(result[i-1], tensors[i]))

    return result

def tensor_product_right(tensors):
    if len(tensors) < 2:
        raise ValueError("At least two tensors are required.")

    result = [tensors[-1]]
    for i in range(len(tensors)-2, -1, -1):
        result.insert(0, np.kron(tensors[i], result[0]))

    return result

def multiply_right_matrices(matrices):
    if len(matrices) < 2:
        raise ValueError("At least two matrices are required.")

    result = [matrices[-1]]
    for i in range(len(matrices)-2, -1, -1):
        result.insert(0, np.matmul(matrices[i], result[0]))
        # Alternatively, you can use the '@' operator:
        # result.insert(0, matrices[i] @ result[0])

    return result

def Series_matrix_multiply(matrix_lst):
    return reduce(lambda a,b:np.matmul(a,b),matrix_lst)

In [14]:
# CUE parameter generator function -

def CUE_parameter_generator(size,delta):
    '''
    input: 
    size = N (size of one side of the random unitary matrix 
    being produced), depending on which, this function return a parameter 
    array of size. 
    
    delta = another user defined variable for determining random unitary
    
    return:
    Returns an array of size (n-1)^2 with the structure -
    [alpha,
    
    phi(1,2),
    psi(1,2),
    chi(1,2),
    
    phi(2,3),
    psi(2,3),
    phi(1,3),
    psi(1,3),
    chi(1,3),
    .
    .
    .
    phi(1,n),
    psi(1,n),
    chi(1,n),
    ]
    '''
    
    parameters = []
    parameters.append(('alpha',0,1,generate_alpha(delta)))
    for s in range(2,size+1):
        for r in range(s-1,0,-1):
            Omega = random.uniform(0,delta)
            parameters.append(('Omega',r,s,generate_Omega(delta)))
            parameters.append(('psi',r,s,generate_psi(r,s,delta)))
            if r==1:
                parameters.append(('chi',r,s,generate_chi(r,s,delta)))
    return parameters

delta = 1
size = 3
parameters = np.array(CUE_parameter_generator(size,delta))
parameters

array([['alpha', '0', '1', '2.627435926794773'],
       ['Omega', '1', '2', '0.33140467037240295'],
       ['psi', '1', '2', '0.13536545334047886'],
       ['chi', '1', '2', '4.654602555775718'],
       ['Omega', '2', '3', '0.36481173270242595'],
       ['psi', '2', '3', '3.211800888732783'],
       ['Omega', '1', '3', '0.32186433361455613'],
       ['psi', '1', '3', '0.8443472525188042'],
       ['chi', '1', '3', '2.7214972118897816']], dtype='<U32')

In [15]:
def Elementary_Unitary_Matrices_Generator(size,r,s,delta,Omega, psi, chi):
    # Creating the matrix
    E = np.eye(size, dtype=complex)
    r-=1 
    s-=1
    
    phi = math.asin(Omega**(1/(2*(r+1))))
    
    E[r,r] = cmath.cos(phi) * cmath.exp(complex(0,psi))  
    E[r,s] = cmath.sin(phi) * cmath.exp(complex(0,chi))
    E[s,r] = -cmath.sin(phi) * cmath.exp(complex(0,-chi))
    E[s,s] = cmath.cos(phi) * cmath.exp(complex(0,-psi))
    
    return E

n=3
a=1
b=3
delta = 1
Omega = generate_Omega(delta)
psi = generate_psi(a,b,delta)
chi = generate_chi(a,b,delta)
    
print(Elementary_Unitary_Matrices_Generator(n,a,b,delta,Omega,psi,chi))

[[-0.17646647-0.87526426j  0.        +0.j          0.37492195-0.24941049j]
 [ 0.        +0.j          1.        +0.j          0.        +0.j        ]
 [-0.37492195-0.24941049j  0.        +0.j         -0.17646647+0.87526426j]]


In [16]:
# CUE_generator - 

# First step is to work with unitary matrices. 
# It will have a function to calculate random unitary matrix using Zyczkowski's method.


def CUE_generator(size, parameters, delta):
    '''
    input: 
    
    1) size = it is the dimensionality (of one side) of the aimed 
    random unitary matrix.
    
    2) parameters = This is a list of parameters used to parametrize the 
    unitary matrix, necessary to generate CUE(Circular Unitary Ensembles)
    deterministically. It is crucial to keep track of these parameters to
    eventually work with gradient descent.
    
    return:
    1) return a unitary matrix of given size, with given parameters and 
    delta.
    '''
    alpha = float(parameters[0][3])
    
    Left = []
    Elementary_matrices = [cmath.exp(complex(0,alpha))]
    right = []
    
    for s in range(2,size+1):
        for r in range(s-1,0,-1):
            indexes = para_i(r,s)
            Omega = float(parameters[indexes[0]][3])
            psi = float(parameters[indexes[1]][3])
            if r==1:
                chi = float(parameters[indexes[2]][3])
            else:
                chi = 0
            E = Elementary_Unitary_Matrices_Generator(size,r,s,delta,Omega,psi,chi)

            if not Qobj(E).check_isunitary():
                raise Exception('U is not unitary.')

            Elementary_matrices.append(E)
    
    
    Left_multiply = Series_multiply(Elementary_matrices,'left')
    Right_multiply = Series_multiply(Elementary_matrices,'right')
    U = Series_multiply(Elementary_matrices)
    
    if Qobj(U).check_isunitary():
        return (U,Elementary_matrices, Left_multiply, Right_multiply)
    else:
        raise Exception('U is not unitary.')
    
    
delta = 1
size = 3
parameters = CUE_parameter_generator(size, delta)

U = CUE_generator(size, parameters, delta)
U

(array([[ 0.66099077-0.22755394j,  0.02862814-0.29593163j,
         -0.29512148+0.57949859j],
        [-0.6182499 -0.25753091j,  0.2407972 +0.13073998j,
         -0.58971918+0.35860835j],
        [-0.25035029-0.00894563j, -0.35444983-0.84314529j,
         -0.20998588-0.23795372j]]),
 [(0.38753805669143887+0.9218537056473891j),
  array([[ 0.40646125+0.54226528j, -0.71387736-0.17639936j,
           0.        +0.j        ],
         [ 0.71387736-0.17639936j,  0.40646125-0.54226528j,
           0.        +0.j        ],
         [ 0.        +0.j        ,  0.        +0.j        ,
           1.        +0.j        ]]),
  array([[ 1.        +0.j        ,  0.        +0.j        ,
           0.        +0.j        ],
         [ 0.        +0.j        ,  0.39153065+0.10087162j,
           0.91461941+0.j        ],
         [ 0.        +0.j        , -0.91461941-0.j        ,
           0.39153065-0.10087162j]]),
  array([[-0.78283901-0.05719577j,  0.        +0.j        ,
           0.3923947 +0.4794977

In [17]:
# Calculating Y
def Y_Calculator(sep_state_set,U,rho):
    '''
    Inputs: 
    sep_state_set = list of seperable state vectors
    U_lst = the unitary to be applied for optimization
    rho = the state of the system being tested for entanglement, given as a density matrix
    
    Outputs:
    (Y,Y_list,left_Y, right_Y)
    where
    
    Y = The final calcuated value of Y, given value inputs.
    
    Y_list = The list of 'Y multiplier' 
    
    left_Y = The list of multiplied <Y(i,j)> from the left - 
    [(<Y(1,1)>), (<Y(1,1)><Y(1,2)>),....,(<Y(1,1)>...<Y(N,N)>)]
    
    right_Y = The list of multiplied <Y(i,j)> from the right -
    [(<Y(N,N)>), (<Y(N,N)><Y(N-1,N)>),....,(<Y(N,N)>...<Y(1,1)>)]
    
    '''
    U = Qobj(U)
    U_dag = U.dag()
    
    system_dim = len(sep_state_set)
    
    Y_list = []
    Yij_list = []
    Yij_matrix = []
    
    for i in range(system_dim):
        Yij_matrix.append([])
        for j in range(system_dim):
            Y_multiplier = reduce(np.matmul,[np.array(sep_state_set[i].dag().full()),np.array(U_dag.full()),np.array(rho.full()),np.array(U.full()),np.array(sep_state_set[j].full())])
            Yij_list.append([Y_multiplier[0][0],i,j])
            Yij_matrix[i].append(Y_multiplier[0][0])
            Y_list.append(Y_multiplier[0][0])
    
    Y = reduce(lambda a,b:a*b, Y_list)
    left_Y = Series_multiply(Y_list,'left')
    right_Y = Series_multiply(Y_list,'right')
    
    return (Y,Yij_list,left_Y,right_Y,Yij_matrix)


#inputs:
system = [3,3]        
Sep_state_set = Seperable_state_set_generator(Mutually_Orthogonal_state_generator(system))

Hilbert_space_size = Series_multiply(system)
rho = rand_dm(Hilbert_space_size)

unitaries = []
Unitary_list = []
Total_U = []
for i in system:
    size = i
    delta = 1
    parameters = CUE_parameter_generator(size, delta)
    CUE = CUE_generator(size, parameters, delta)
    
    Unitary_list.append(CUE)
    Total_U.append(Qobj(CUE[0]))
U = np.array(tensor(Total_U))

Y_Calculator(Sep_state_set, U,rho)

((3.9832552195037005e-12-1.2870184961481653e-27j),
 [[(0.08469823988757395+6.5052130349130266e-18j), 0, 0],
  [(-0.012828666237816663-0.04744137958469109j), 0, 1],
  [(-0.007493842624850035-0.0318278898441991j), 0, 2],
  [(-0.012828666237816663+0.04744137958469109j), 1, 0],
  [(0.15375743936957242-3.469446951953614e-18j), 1, 1],
  [(0.023612834786800443-0.029534026730489257j), 1, 2],
  [(-0.00749384262485004+0.031827889844199105j), 2, 0],
  [(0.023612834786800454+0.029534026730489257j), 2, 1],
  [(0.08283872324943425-4.4994390158148434e-18j), 2, 2]],
 [(0.08469823988757395+6.5052130349130266e-18j),
  (-0.0010865654504482163-0.004018201348661619j),
  (-0.00011974831940975606+6.469485400720874e-05j),
  (-1.5330019038849402e-06-6.510974165109826e-06j),
  (-2.357104472900278e-07-1.0011107154287264e-06j),
  (-3.5132622479033494e-08-1.667758327579485e-08j),
  (7.940906272256015e-10-9.932180537688737e-10j),
  (4.808445933563931e-11-1.2924697071141057e-26j),
  (3.9832552195037005e-12-1.2870184

In [18]:
def calculate_dEdp(E,para,para_i):
    symbol = para[para_i][0]
    r = int(para[para_i][1])
    s = int(para[para_i][2])
    parameter_value = float(para[para_i][3])
    
    si = np.array(E).shape
    
    chi=0
    dEdp = np.zeros(E.shape, dtype=complex)
    r-=1 
    s-=1
        
    if symbol == 'Omega':
        Omega = parameter_value
        psi = float(para[para_i+1][3])
        if r==1:
            chi = float(para[para_i+2][3])
#         print(Omega, r)
        
        power = 1/(2*(r+1))
        z = Omega**(power)
        
        phi = math.asin(z)
        dphi_dOmega = (1/math.sqrt(1-z**2))*(power)*(Omega**(power-1))

        dEdp[r,r] = -cmath.sin(phi) * cmath.exp(complex(0,psi)) * dphi_dOmega
        dEdp[r,s] = cmath.cos(phi) * cmath.exp(complex(0,chi)) * dphi_dOmega
        dEdp[s,r] = -cmath.cos(phi) * cmath.exp(complex(0,-chi)) * dphi_dOmega
        dEdp[s,s] = -cmath.sin(phi) * cmath.exp(complex(0,-psi)) * dphi_dOmega
        
    elif symbol == 'psi':
        Omega = float(para[para_i-1][3])
        psi = parameter_value
        if r==1:
            chi = float(para[para_i+1][3])
        
        phi = math.asin(Omega**(1/(2*(r+1))))
    
        dEdp[r,r] = complex(0,1) * cmath.cos(phi) * cmath.exp(complex(0,psi))  
        dEdp[r,s] = 0
        dEdp[s,r] = 0
        dEdp[s,s] = -complex(0,1) * cmath.cos(phi) * cmath.exp(complex(0,-psi))
    
    elif symbol == 'chi':
        Omega = float(para[para_i-2][3])
        psi = float(para[para_i-1][3])
        if r==1:
            chi = parameter_value
        
        phi = math.asin(Omega**(1/(2*(r+1))))
    
        dEdp[r,r] = 0  
        dEdp[r,s] = complex(0,1) * cmath.sin(phi) * cmath.exp(complex(0,chi))
        dEdp[s,r] = -complex(0,1) * -cmath.sin(phi) * cmath.exp(complex(0,-chi))
        dEdp[s,s] = 0
        
    return dEdp

In [19]:
def calculate_du_system_dp(u_system, para,para_i,sys_i):
    symbol = para[para_i][0]
    r = float(para[para_i][1])
    s = float(para[para_i][2])
    parameter_value = float(para[para_i][3])
    
    if symbol=='alpha':
        return complex(0,1)*parameter_value*u_system[0]
    
    elementary_index = int((s-1)*(s)/2 - (r-1))
    E = u_system[1][elementary_index]

    dEdp = calculate_dEdp(E,para,para_i)
    
    if elementary_index == 0:
        du_system_dp = Series_multiply([dEdp,u_system[3][sys_i+1]])
    elif elementary_index == len(u_system[1])-1:
        du_system_dp = Series_multiply([u_system[2][elementary_index-1],dEdp])
    else:
        du_system_dp = Series_multiply([u_system[2][sys_i-1],dEdp,u_system[3][sys_i+1]])
        
    return du_system_dp

In [20]:
def calculate_dUdp(U,para,sys_i,para_i,Total_U,right_tensor_product,left_tensor_product):
    u_system = U[sys_i]
    du_system_dp = calculate_du_system_dp(u_system, para,para_i,sys_i)
    
    if sys_i == 0:
        dUdp = tensor_product([du_system_dp,right_tensor_product[sys_i+1]])
    elif sys_i == len(U)-1:
        dUdp = tensor_product([left_tensor_product[sys_i-1],du_system_dp])
    else:
        dUdp = tensor_product([left_tensor_product[sys_i-1],du_system_dp,right_tensor_product[sys_i+1]])
    
    return dUdp

In [21]:
def calculate_Yij_dp_diagonal(sep_state_set,rho,U,para,sys_i,para_i,xi_a,xi_b,Total_U,right_tensor_product,left_tensor_product):
    Total_U_dag = np.array(Total_U).transpose().conjugate()
    xi_a_dag = np.array(xi_a).transpose().conjugate()
      
    dUdp = calculate_dUdp(U,para,sys_i,para_i,Total_U,right_tensor_product,left_tensor_product)
    dUdp_dag = np.array(dUdp).transpose().conjugate()
    
    dYij_dp = Series_matrix_multiply([xi_a_dag,dUdp_dag,np.array(rho),Total_U,np.array(xi_b)])
    
    return dYij_dp[0][0]

def calculate_Yij_dp_off_diagonal(sep_state_set,rho,U,para,sys_i,para_i,xi_a,xi_b,Total_U,right_tensor_product,left_tensor_product):
    Total_U_dag = np.array(Total_U).transpose().conjugate()
    xi_a_dag = np.array(xi_a).transpose().conjugate()
      
    dUdp = calculate_dUdp(U,para,sys_i,para_i,Total_U,right_tensor_product,left_tensor_product)
    dUdp_dag = np.array(dUdp).transpose().conjugate()
    
    term_a = Series_matrix_multiply([xi_a_dag,dUdp_dag,np.array(rho),Total_U,np.array(xi_b)])
    term_b = Series_matrix_multiply([xi_a_dag,Total_U_dag,np.array(rho),dUdp,np.array(xi_b)])
    
    dYij_dp = term_a + term_b
    
    return dYij_dp[0][0]

def calculate_dYij_dp(sep_state_set,rho,U,para,sys_i,para_i,xi_a,xi_b,Total_U,right_tensor_product,left_tensor_product):
    Total_U_dag = np.array(Total_U).transpose().conjugate()
    xi_a_dag = np.array(xi_a).transpose().conjugate()
      
    dUdp = calculate_dUdp(U,para,sys_i,para_i,Total_U,right_tensor_product,left_tensor_product)
    
    b = (-Series_matrix_multiply([dUdp,Total_U_dag,np.array(rho),Total_U])+np.matmul(np.array(rho),dUdp))
    a = [Total_U_dag,b]
    Ut = Series_matrix_multiply(a)
    
    dYij_dp = Series_matrix_multiply([xi_a_dag,Ut,np.array(xi_b)])
    return dYij_dp[0][0]


In [22]:
def calculate_dY_dp(sep_state_set,rho,U,para,Y,sys_i,para_i,Total_U,right_tensor_product,left_tensor_product):
    partial_diff_sum = 0
    for y in Y[1]:
        xi_a = sep_state_set[y[1]]
        xi_b = sep_state_set[y[2]]
        
        dYij_dp = calculate_dYij_dp(sep_state_set,rho,U,para,sys_i,para_i,xi_a,xi_b,Total_U,right_tensor_product,left_tensor_product)
        partial_diff_sum+=((dYij_dp)/(y[0]))
    
    # N calculator
    N = (len(sep_state_set))**2
    
    #Y 
    Y = Y[0]
    
    dYdp = partial_diff_sum*(Y**(1/N))/N
    return dYdp

def calculate_dY_dp_modified(sep_state_set,rho,U,para,Y,sys_i,para_i,Total_U,right_tensor_product,left_tensor_product):
    # Make Y[1] easier to access by making it into a matrix rather than a list.
    y = Y[4]
    
    partial_diff_sum = 0
    abs = lambda f:cmath.sqrt(f.real**2+f.imag**2)
    
    for i in range(len(y)):
        for j in range(len(y[i])):
            xi_a = sep_state_set[i]
            xi_b = sep_state_set[j]

            if i==j:
                y_original = y[i][j]
                dYij_dp = calculate_Yij_dp_diagonal(sep_state_set,rho,U,para,sys_i,para_i,xi_a,xi_b,Total_U,right_tensor_product,left_tensor_product)
                dYij_dp = 2*dYij_dp.real
                partial_diff_sum+=((dYij_dp)/abs(y_original))
            elif i<j:
                Yij = y[i][j]
                Yji = y[j][i]
                y_original = Yij*Yji
    
                dYij_dp = calculate_Yij_dp_off_diagonal(sep_state_set,rho,U,para,sys_i,para_i,xi_a,xi_b,Total_U,right_tensor_product,left_tensor_product)
                dYij_dp = dYij_dp*Yji
                dYij_dp = 2*dYij_dp.real

                partial_diff_sum+=((dYij_dp)/abs(y_original))
            else:
                continue
    
    dYdp = abs(Y[0])*partial_diff_sum
    return dYdp.real

In [23]:
def Gradient(sep_state_set,rho,U,parameters):
    '''
    U is a list of local unitaries = [U_1,U_2,....,U_k]
    '''
    Local_unitary_list = [U[n][0] for n in range(len(U))]
    
    Total_U = tensor_product(Local_unitary_list)
    right_tensor_product = tensor_product_right(Local_unitary_list)
    left_tensor_product = tensor_product_left(Local_unitary_list)
    
    Y = Y_Calculator(Sep_state_set, Total_U, rho)
    
#     abs = lambda f:cmath.sqrt(f.real**2+f.imag**2)
#     print(Y[0],abs(Y[0]).real)
    gradient = []
    for sys_i in range(len(parameters)):
        gradient.append([])
        for para_i in range(len(parameters[sys_i])):
#             r = parameters[sys_i][para_i][1]
#             s = parameters[sys_i][para_i][2]
            para = parameters[sys_i]
            dYdp = calculate_dY_dp_modified(sep_state_set,rho,U,para,Y,sys_i,para_i,Total_U,right_tensor_product,left_tensor_product)
            gradient[sys_i].append(dYdp)
            
    return gradient

#inputs:
system = [3,3]        
sep_state_set = Seperable_state_set_generator(Mutually_Orthogonal_state_generator(system))

Hilbert_space_size = Series_multiply(system)
rho = rand_dm(Hilbert_space_size)

unitaries = []
delta = 1
parameters = []
for n in system:
    size = n
    
    p = CUE_parameter_generator(size, delta)
    parameters.append(p)
    
    uni = CUE_generator(size, p, delta)
    unitaries.append(uni)
    
Gradient(sep_state_set,rho,unitaries,parameters)

[[9.786847444872995e-27,
  -1.1784921307795243e-10,
  -5.692993881753465e-11,
  6.727601423773777e-11,
  -1.089565549732304e-10,
  -1.9409721842190837e-11,
  -2.0071720708827354e-10,
  9.711009992064426e-12,
  8.838579596448753e-11],
 [-7.088626792265121e-26,
  5.463251092039599e-11,
  2.822160633527564e-11,
  -4.6002445042611614e-11,
  -9.8879918121777e-11,
  -8.403845242640408e-11,
  -2.4324411760721814e-10,
  9.086076687445288e-12,
  1.2226687888340072e-10]]

In [29]:
def min_max_para(para,delta):
    symbol = para[0]
    if symbol=='alpha':
        para_min = 0
        para_max = 2*math.pi*delta
    elif symbol == 'Omega':
        para_min = sys.float_info.epsilon
        para_max = delta
        if para_max == 1:
            para_max = para_max-sys.float_info.epsilon
    elif symbol == 'psi':
        para_min = 0
        para_max = 2*math.pi*delta
    elif symbol == 'chi':
        para_min = 0
        para_max = 2*math.pi*delta
    
    return para_min,para_max

def adjusted_parameters(parameters, gradient, stepsize,delta):
    new_parameters = []
    for sys in range(len(parameters)):
        new_parameters.append([])
        for para in range(len(parameters[sys])):
            a = parameters[sys][para][3]
            b = stepsize*gradient[sys][para]
            old_val = list(parameters[sys][para])
            
            new_val = []
            for i in range(len(old_val)-1):
                new_val.append(old_val[i])
                
            para_min,para_max = min_max_para(new_val,delta)
            new_val.append(max(para_min,min(para_max,a+b)))
            
            new_parameters[sys].append(tuple(new_val))
            
    return new_parameters

def parameter_based_unitary(parameters,delta):
    U = []
    for p in parameters:
        size = int(math.sqrt(len(p)))
        
        uni = CUE_generator(size, p, delta)
        U.append(uni)
        
    return U


def Y_calculator_wrapper(sep_state_set, rho, U):
    
    Local_unitary_list = [U[n][0] for n in range(len(U))]
    
    Total_U = tensor_product(Local_unitary_list)
    right_tensor_product = tensor_product_right(Local_unitary_list)
    left_tensor_product = tensor_product_left(Local_unitary_list)
    
    Y = Y_Calculator(Sep_state_set, Total_U, rho)
    return Y

def dynamic_stepsize():
    stepsize = 0.1
    return stepsize


In [44]:
def Gradient_Descent(system,rho,delta):
    abs = lambda f:cmath.sqrt(f.real**2+f.imag**2)
    sep_state_set = Seperable_state_set_generator(Mutually_Orthogonal_state_generator(system))

    parameters = []
    
    for n in system:
        size = n
        p = CUE_parameter_generator(size, delta)
        parameters.append(p)
        
    U = parameter_based_unitary(parameters,delta)
    
    Y = Y_calculator_wrapper(sep_state_set, rho, U)
    Y_old = abs(Y[0]).real
    Y_correction = [Y_old]
    
    Y_correction_max = max(Y_correction)

    while Y_correction_max==Y_correction[-1]:
        print(Y_correction_max)
        gradient = Gradient(sep_state_set,rho,unitaries,parameters) 
        nrm = np.linalg.norm(np.array(gradient))
        gradient = gradient/nrm
     
        stepsize = dynamic_stepsize()
        new_parameters = adjusted_parameters(parameters, gradient, stepsize,delta)

        U_new = parameter_based_unitary(new_parameters,delta)
        Y_new = Y_calculator_wrapper(sep_state_set, rho, U_new)
        Y_new = abs(Y_new[0]).real
        
        Y_correction.append(Y_new)
        
        Y_correction_max = max(Y_correction_max,Y_new)
        
        parameters = new_parameters
    
    return Y_correction

system = [3,3]
# rho = rand_dm(Hilbert_space_size)
Delta = 1
GD = Gradient_Descent(system,rho,Delta)

xpoints = np.array(list(range(len(GD))))
ypoints = np.array(GD)

plt.plot(xpoints, ypoints, '--ro')
plt.show()

print(GD[-2])

2.2333582879876003e-12
3.644795547526215e-12
3.644795747616377e-12
3.644795947706558e-12
3.644796147796783e-12
3.644796347887031e-12
3.6447965479773096e-12
3.644796748067644e-12
3.644796948157987e-12
3.64479714824837e-12
3.644797348338804e-12
3.6447975484292714e-12
3.6447977485197525e-12
3.644797948610286e-12
3.644798148700828e-12
3.6447983487914365e-12
3.6447985488820645e-12
3.644798748972726e-12
3.6447989490634185e-12
3.644799149154136e-12
3.644799349244897e-12
3.644799549335702e-12
3.644799749426532e-12
3.6447999495174095e-12
3.644800149608301e-12
3.644800349699246e-12
3.644800549790197e-12
3.644800749881218e-12
3.644800949972251e-12
3.644801150063316e-12
3.6448013501544234e-12
3.644801550245563e-12
3.644801750336736e-12
3.644801950427943e-12
3.644802150519205e-12
3.644802350610482e-12
3.644802550701775e-12
3.644802750793143e-12
3.64480295088451e-12
3.644803150975937e-12
3.6448033510673606e-12
3.644803551158849e-12
3.644803751250352e-12
3.644803951341883e-12
3.644804151433469e-12
3.

KeyboardInterrupt: 

In [26]:
rho = rand_dm(Hilbert_space_size)
rho

Quantum object: dims = [[9], [9]], shape = (9, 9), type = oper, isherm = True
Qobj data =
[[ 0.17067326+0.j         -0.06679523-0.05437948j -0.01526646+0.00222932j
   0.00547742-0.01471044j  0.03042479-0.01324357j  0.00944024-0.03684864j
   0.0603216 -0.00963774j  0.01712868+0.02158257j  0.00261143-0.01000834j]
 [-0.06679523+0.05437948j  0.11719439+0.j          0.03676175-0.00750743j
  -0.00641379+0.05099778j  0.02927399+0.00091055j  0.03625887+0.00930143j
  -0.06483907+0.05485j     0.00290278-0.02398674j  0.00542767+0.01994499j]
 [-0.01526646-0.00222932j  0.03676175+0.00750743j  0.08734179+0.j
  -0.0054019 +0.04238219j -0.01179123-0.01543896j  0.00540785+0.01441413j
   0.00483243+0.01037363j -0.01096221-0.01445478j  0.00544481+0.01064347j]
 [ 0.00547742+0.01471044j -0.00641379-0.05099778j -0.0054019 -0.04238219j
   0.0913925 +0.j         -0.01887813-0.0129454j  -0.02547631-0.02013545j
   0.01789838+0.04568398j -0.02067404-0.0288669j   0.03074625-0.00839074j]
 [ 0.03042479+0.01324357j 

In [43]:
def finding_max_collectibility(system,rho,delta,iterations):
    max_Y = []
    Compute_times = []
    for i in range(iterations):
#         if i%100==0:print(i) 
        print(i)
        s1 = time.time()
        max_Y.append(Gradient_Descent(system,rho,delta)[-2])
        s2 = time.time()
        Compute_times.append(s2-s1)
    return max_Y,Compute_times


system = [3,3]
# rho = rand_dm(Hilbert_space_size)
Delta = 1
iterations = 100
print(rho)
max_Y,Compute_times = finding_max_collectibility(system,rho,delta,iterations)

max_Y = sorted(max_Y)

xpoints = np.array(list(range(len(max_Y))))
ypoints = np.array(max_Y)

plt.plot(xpoints, ypoints, '--ro')
plt.show()
print(f'Maximum collectibility = {max(max_Y)}')

xpoints1 = np.array(list(range(len(Compute_times))))
ypoints1 = np.array(Compute_times)

plt.plot(xpoints1, ypoints1, '--bo')
plt.show()
print(f'Total Compute time = {sum(Compute_times)}')


Quantum object: dims = [[9], [9]], shape = (9, 9), type = oper, isherm = True
Qobj data =
[[ 0.17067326+0.j         -0.06679523-0.05437948j -0.01526646+0.00222932j
   0.00547742-0.01471044j  0.03042479-0.01324357j  0.00944024-0.03684864j
   0.0603216 -0.00963774j  0.01712868+0.02158257j  0.00261143-0.01000834j]
 [-0.06679523+0.05437948j  0.11719439+0.j          0.03676175-0.00750743j
  -0.00641379+0.05099778j  0.02927399+0.00091055j  0.03625887+0.00930143j
  -0.06483907+0.05485j     0.00290278-0.02398674j  0.00542767+0.01994499j]
 [-0.01526646-0.00222932j  0.03676175+0.00750743j  0.08734179+0.j
  -0.0054019 +0.04238219j -0.01179123-0.01543896j  0.00540785+0.01441413j
   0.00483243+0.01037363j -0.01096221-0.01445478j  0.00544481+0.01064347j]
 [ 0.00547742+0.01471044j -0.00641379-0.05099778j -0.0054019 -0.04238219j
   0.0913925 +0.j         -0.01887813-0.0129454j  -0.02547631-0.02013545j
   0.01789838+0.04568398j -0.02067404-0.0288669j   0.03074625-0.00839074j]
 [ 0.03042479+0.01324357j 

7.823690523098396e-12
7.82369059068569e-12
7.823690658273019e-12
7.823690725860325e-12
7.823690793447644e-12
7.823690861034946e-12
7.823690928622278e-12
7.823690996209593e-12
7.823691063796883e-12
7.823691131384189e-12
7.823691198971503e-12
7.823691266558778e-12
7.823691334146106e-12
7.823691401733386e-12
7.823691469320698e-12
7.823691536907988e-12
7.823691604495244e-12
7.823691672082556e-12
7.823691739669855e-12
7.82369180725711e-12
7.823691874844417e-12
7.823691942431682e-12
7.823692010018975e-12
7.82369207760623e-12
7.823692145193529e-12
7.823692212780805e-12
7.823692280368092e-12
7.823692347955335e-12
7.823692415542597e-12
7.823692483129883e-12
7.823692550717144e-12
7.823692618304414e-12
7.823692685891655e-12
7.823692753478938e-12
7.823692821066181e-12
7.823692888653437e-12
7.823692956240699e-12
7.823693023827953e-12
7.823693091415215e-12
7.823693159002458e-12
7.823693226589707e-12
7.823693294176949e-12
7.823693361764172e-12
7.82369342935144e-12
7.823693496938687e-12
7.823693564525

7.823716611666244e-12
7.823716679252829e-12
7.823716746839456e-12
7.823716814426053e-12
7.823716882012667e-12
7.823716949599214e-12
7.823717017185842e-12
7.823717084772465e-12
7.823717152359044e-12
7.823717219945603e-12
7.823717287532204e-12
7.8237173551188e-12
7.823717422705386e-12
7.823717490291962e-12
7.823717557878566e-12
7.823717625465157e-12
7.823717693051742e-12
7.823717760638308e-12
7.823717828224886e-12
7.823717895811438e-12
7.823717963398021e-12
7.823718030984597e-12
7.823718098571139e-12
7.823718166157719e-12
7.823718233744232e-12
7.823718301330843e-12
7.823718368917394e-12
7.823718436503945e-12
7.823718504090501e-12
7.823718571677074e-12
7.823718639263606e-12
7.823718706850152e-12
7.823718774436692e-12
7.823718842023248e-12
7.8237189096098e-12
7.82371897719631e-12
7.823719044782881e-12
7.823719112369397e-12
7.82371917995593e-12
7.82371924754246e-12
7.823719315129058e-12
7.823719382715548e-12
7.82371945030208e-12
7.823719517888597e-12
7.823719585475158e-12
7.823719653061654e

7.823742564788168e-12
7.82374263237405e-12
7.823742699959973e-12
7.823742767545844e-12
7.823742835131709e-12
7.823742902717628e-12
7.823742970303493e-12
7.823743037889403e-12
7.823743105475311e-12
7.823743173061144e-12
7.823743240647028e-12
7.823743308232975e-12
7.823743375818838e-12
7.82374344340473e-12
7.823743510990595e-12
7.823743578576458e-12
7.823743646162348e-12
7.823743713748215e-12
7.823743781334092e-12
7.823743848919957e-12
7.82374391650586e-12
7.823743984091709e-12
7.82374405167757e-12
7.823744119263439e-12
7.82374418684929e-12
7.823744254435147e-12
7.823744322021021e-12
7.823744389606863e-12
7.823744457192714e-12
7.823744524778581e-12
7.823744592364417e-12
7.823744659950274e-12
7.82374472753612e-12
7.823744795121966e-12
7.823744862707795e-12
7.823744930293611e-12
7.82374499787948e-12
7.823745065465317e-12
7.823745133051118e-12
7.823745200636993e-12
7.823745268222819e-12
7.82374533580861e-12
7.823745403394467e-12
7.823745470980296e-12
7.82374553856611e-12
7.823745606151896e-

KeyboardInterrupt: 

In [ ]:
def scaling_analysis(system, rho,delta,order_range,scaling_factor):
    max_max_Y = []
    compute = []
    for i in range(order_range):
        iterations = scaling_factor**i
        
        max_Y,Compute_times = finding_max_collectibility(system,rho,delta,iterations)
        
        max_Y = sorted(max_Y)
        
        xpoints = np.array(list(range(len(max_Y))))
        ypoints = np.array(max_Y)
        plt.plot(xpoints, ypoints, '--ro')
        plt.show()
        print(f'Maximum collectibility = {max(max_Y)}')

        xpoints1 = np.array(list(range(len(Compute_times))))
        ypoints1 = np.array(sorted(Compute_times))
#         plt.plot(xpoints1, ypoints1, '--bo')
#         plt.show()
        print(f'For iterations = {iterations},Total Compute time = {sum(Compute_times)}')
        
        max_max_Y.append(max(max_Y))
        compute.append(sum(Compute_times))
    return max_max_Y,compute

system = [3,3]
# rho = rand_dm(Hilbert_space_size)
Delta = 1
order_range = 14
scaling_factor = 2
max_max_Y,compute = scaling_analysis(system, rho,delta,order_range,scaling_factor)

xpoints = np.array(list(range(len(max_max_Y))))
ypoints = np.array(max_max_Y)

plt.plot(xpoints, ypoints, '--ro')
plt.show()
print(f'Maximum collectibility = {max(max_max_Y)}')

xpoints1 = np.array(list(range(len(compute))))
ypoints1 = np.array(compute)

plt.plot(xpoints1, ypoints1, '--bo')
plt.show()
print(f'Total Compute time = {sum(compute)}')


In [124]:
math.pi

3.141592653589793

In [142]:
math.log(2)

0.6931471805599453

In [368]:
a = [1,2,3,4]
a[:-1]

[1, 2, 3]